<a href="https://colab.research.google.com/github/iggyloh/Tesla-and-amazon-stock-dashboard/blob/main/DAC_proj_2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# Path to the uploaded file
file_path = "/content/updated_papers.csv" ## change ur file path

# Read the CSV into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())
test_df = df.loc[df['year'] == '2000'].head(10)
test_df

In [1]:
#Uncomment once and run the file to get access to all the necessary libraries
%pip install -r requirements.txt
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libmagic1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (314 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [2]:
import os
import bs4
import ast
import uuid
import json
import base64
import markdown
from PIL import Image
from pprint import pprint
from PyPDF2 import PdfMerger
from base64 import b64decode
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from IPython.display import Image, display
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Text, Image
from typing_extensions import Annotated, TypedDict, Sequence

from langchain import hub
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.text_splitter  import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

from langchain_community.document_loaders import(
  PyPDFLoader,
  Docx2txtLoader,
  UnstructuredPDFLoader,
  WebBaseLoader,
  UnstructuredMarkdownLoader,
  UnstructuredWordDocumentLoader,
  TextLoader,
  UnstructuredPDFLoader
)

from langchain.schema import BaseMessage
from langchain_community.vectorstores import Chroma, InMemoryVectorStore
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_history_aware_retriever
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

from IPython.display import Image, display
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

import gradio as gr

In [3]:


# Tracing via Langsmith
trace = os.getenv("LANGCHAIN_TRACING_V2")
langsmith = os.getenv("LANGCHAIN_API_KEY")

load_dotenv('template.env')

# Build a GPT model
gpt = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0.1)

# Embedding model
embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-large',
    openai_api_keys = os.getenv('OPENAI_API_KEY')
)

# Small test
response = gpt.invoke('Who won the most league world championships')
print(response.content)

<ipython-input-3-6d8f34523f58>:13: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
/usr/local/lib/python3.10/dist-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! openai_api_keys is not default parameter.
                    openai_api_keys was transferred to model_kwargs.
                    Please confirm that openai_api_keys is what you intended.
  warnings.warn(


As of my last update, the New York Yankees hold the record for the most World Series championships in Major League Baseball (MLB), having won 27 titles. If you are referring to a different league or sport, please provide more details so I can assist you accurately.


In [12]:
import torch

In [17]:
## trying from langchain website
#embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,       # Maximum number of characters per chunk
    chunk_overlap=200,    # Number of overlapping characters between chunks
    separators=["\n\n", "\n", " ", ""]  # Order of separators to try splitting
)

from transformers import AutoTokenizer, AutoModel
import torch

def get_embedding(text):
  inputs = tokenizer(text, return_tensors = 'pt', padding = True, truncation = True)
  with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state.mean(dim = 1)
  return embeddings.squeeze().numpy()

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [24]:
texts = test_df['full_text'].fillna(" ")  # Replace NaN values with empty strings

# Step 2: Convert text into Document objects
documents = [Document(page_content=text) for text in texts]

# Step 3: Split the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = []  # Will store Document objects for chunks

for doc in documents:
    splits = text_splitter.split_text(doc.page_content)  # Split text into chunks
    for chunk in splits:
        all_splits.append(Document(page_content=chunk))  # Store each chunk as a Document object

# Step 4: Add documents to the vector store
document_ids = vector_store.add_documents(documents=all_splits)

# Step 5: Verify the document IDs
print(f"First 3 document IDs: {document_ids[:3]}")


First 3 document IDs: ['743943f0-d29f-4958-99dd-cc5a9b20fc19', '516290a6-8d8b-4839-938d-beaa0d1e6358', '18d5f99a-8026-4a21-9836-a10c2a49958c']


In [37]:
from langchain.embeddings import OpenAIEmbeddings

# Initialize your embedding model (replace with your model of choice)
embedding_model = OpenAIEmbeddings()

# Generate embeddings for the documents
embeddings = embedding_model.embed_documents([doc.page_content for doc in all_splits])

# Store the embeddings in the vector store
for idx, doc in enumerate(all_splits):
    doc.metadata['embedding'] = embeddings[idx]  # Store embedding in metadata

# Now you can add the documents to the vector store with embeddings
document_ids = vector_store.add_documents(documents=all_splits)

# Verify the embeddings are stored
print(f"First 3 document embeddings: {all_splits[:3]}")




First 3 document embeddings: [Document(metadata={'embedding': [-0.01703697407012319, 0.019220066782057815, -0.012456904833577032, 0.013378819168065331, -0.0011735968043799306, 0.027244408970630032, 0.011534990499088732, -0.002439384995642563, -0.03106482075085435, -0.029397999246669325, 0.0003033097827518689, 0.012058637781473444, 0.010148431891361285, 0.008776623570258948, -0.014979261588469685, 0.010974466568818687, 0.01544390679303139, 0.0035106492418390685, -0.012346274889921023, 0.012066012924785999, -0.020046101459515218, 0.009079010965331641, -0.0003072279149481539, -0.06431273235099821, 0.005203283747618042, -0.004384623747811919, 0.017715504018684356, -0.053308765208929305, 0.03133033149804171, 0.002754679590004145, 0.02082788527709728, -0.0158126723405622, -0.027524670004442503, 0.013482073037086227, -0.0074048147834953355, -0.017420490835601665, 0.0027989313812020375, -0.004878769991285126, 0.028837475316399276, -0.014876007719448787, 0.029766765725522688, -0.023394494754510

In [25]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [26]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [27]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [28]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [30]:
llm = ChatOpenAI(model="gpt-4o-mini")
result = graph.invoke({"question": "What is a neural network"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='5273d730-acee-4ccd-bab7-2dae27662577', metadata={}, page_content='Figure  1:  Hand-crafted neural network:  a)  Architecture of the network.  Solid lines  are \nweights, all of strength 1.  Dashed lines indicate modulatory effects.  Neurons  1 through 6 \nare spontaneously active (activity equals  1) under normal conditions. The performance of \nthe network is taken to be the activity of neuron 10.  b) The activation functions of the non(cid:173)\nspontaneous neurons.  The x axis is  the input field and the y axis is the resulting activity of'), Document(id='c22f5a5b-842a-456a-86be-ca8e6b8b8366', metadata={}, page_content='Advances in Neural Information Processing Systems, volume 3, pages 832- 838. Morgan Kauf(cid:173)\nmann, 1991. \n\n[4]  S.  Geman et al.  Neural networks and the bias/variance dilemma.  Neural Computation, 4(1):1-\n\n58,1992.'), Document(id='35fa946c-4319-4dc0-8d5d-d722e6cedebd', metadata={}, page_content='3.1  Hand-Crafted Example \n\nFigure  